In [ ]:
import rospy 
import rospkg 
import sys 
import os
import json
import numpy as np
from transforms3d.quaternions import mat2quat, quat2mat

#st_path = rospkg.RosPack().get_path("scene_setup")
data_dir = "/home/felipe/Documents/isaac_sim_grasping/grasps/grasps_midopen"
print(data_dir)
#out_dir = os.path.join(st_path, "datasets/benchmarking/grasp_data/midopen")
out_dir = "/home/felipe/Documents/isaac_sim_grasping/grasps"
if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)
print(out_dir)

In [ ]:
json_files = [pos_json for pos_json in os.listdir(data_dir) if pos_json.endswith('.json')]

print(json_files)

def isaac_qt_to_rt(qt, trans):
    obj_T = np.eye(4)
    obj_T[:3, :3] = quat2mat(qt)
    obj_T[:3, 3] = trans

    return obj_T

In [ ]:
dofs = None

for j in json_files:
    grasps_path = os.path.join(data_dir,j)
    print(j)
    with open(grasps_path) as fd:
        data = json.load(fd)
       
    n = len(data["pose"])
    print(data["dofs"][0])
    
    for i in range(n):
        
        pose = data["pose"][i]
        dofs = data["dofs"][i]
        rot = pose[3:]
        trans = pose[:3]
        #print(pose)
        # NOTE: here qt is in isaac format
        RT = isaac_qt_to_rt(rot, trans)
        
        T = np.eye(4)
        #T[0,3] = 0.19645 #finger_tip_link offset
        T[1,3] = (dofs[1]-dofs[0])/2 # Center Fingers offset
        
        new_RT = np.matmul(RT,T)
        
        quat = mat2quat(new_RT[:3,:3])

        trans = new_RT[:3, 3]

        pose = np.concatenate((trans,quat))
        
        #print(pose)
        
        data["pose"][i] = pose.tolist()
        print(dofs)

    
    output_path = os.path.join(out_dir,j)
    with open(output_path,'w') as outfile:
        #json.dump(data,outfile)
        pass
    
        
    

In [ ]:
print(dofs[1])